In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import date

sys.path.append('../')

from util.chart_methods import draw_investment_log
from util.math_methods import round_down

In [ ]:
policy_id = 2
portfolio_name = 'test'
df = pd.read_csv('../logs/{0}-{1}.log'.format(portfolio_name, policy_id), parse_dates=['date'], date_parser=pd.to_datetime)

In [ ]:
draw_investment_log(df, xlabel='Trade Date', ylabel='Asset (unit: 10K)', title='Assets Changes Table')

In [ ]:
start_date = date.fromtimestamp(df.date.values[0].astype(int) / 1e9)
end_date = date.fromtimestamp(df.date.values[-1].astype(int) / 1e9)
years = end_date.year - start_date.year if end_date.year > start_date.year else 1
initial_total = df.total.values[0]
total = df.total.values[-1]
cagr = round_down((total / initial_total) ** (1 / years) - 1)
print('CAGR: {0}'.format(cagr))

In [ ]:
df_trade = pd.read_csv('../logs/trade_{0}-{1}.log'.format(portfolio_name, policy_id), parse_dates=['date','hold_date'], date_parser=pd.to_datetime)
df_trade['status'] = df_trade['benefit'].apply(lambda benefit: 'win' if benefit > 0 else 'loss')

In [ ]:
df_trade[['ts_code','status']].groupby(['status']).count().rename(columns={"ts_code": "count"})

In [ ]:
df_benefit = df_trade[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "sum"})
df_win = df_trade[df_trade['status']=='win']
df_win_sum = df_win[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "win"})
df_win_count = df_win[['ts_code', 'benefit']].groupby(['ts_code']).count().rename(columns={"benefit": "win_count"})
df_loss = df_trade[df_trade['status']=='loss']
df_loss_sum = df_loss[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "loss"})
df_loss_count = df_loss[['ts_code', 'benefit']].groupby(['ts_code']).count().rename(columns={"benefit": "loss_count"})

In [ ]:
df_benefit.join(df_win_sum).join(df_win_count).join(df_loss_sum).join(df_loss_count).sort_values(by=['sum'], ascending=False)

In [ ]:
df_trade.tail(60)